### 1. Load Required Library
#### Purpose: Loads the stats library, which contains essential statistical functions, including dist() for calculating distance matrices.

In [2]:
library(stats)

### 2. Generate Feature × Sample Matrix (F × S)

In [3]:
set.seed(123)
F <- 5   # Number of features
S <- 6   # Number of samples (3 in R, 3 in G)
feature_sample_matrix <- matrix(runif(F * S, 1, 10), nrow = F)

# Assign classes to samples
classes <- c("R", "R", "R", "G", "G", "G")  # First 3 Red, next 3 Green

cat("\nFeature-Sample Matrix (F x S):\n")
print(feature_sample_matrix)

# Transpose for samples as rows
sample_feature_matrix <- t(feature_sample_matrix)

cat("\nSample-Feature Matrix (S x F):\n")
print(sample_feature_matrix)


Feature-Sample Matrix (F x S):
         [,1]     [,2]     [,3]     [,4]     [,5]     [,6]
[1,] 3.588198 1.410008 9.611500 9.098425 9.005854 7.376774
[2,] 8.094746 5.752949 5.080007 3.214790 7.235231 5.896594
[3,] 4.680792 9.031771 7.098136 1.378536 6.764561 6.347278
[4,] 8.947157 5.962915 6.153701 3.951286 9.948428 3.602438
[5,] 9.464206 5.109533 1.926322 9.590533 6.901352 2.324023

Sample-Feature Matrix (S x F):
         [,1]     [,2]     [,3]     [,4]     [,5]
[1,] 3.588198 8.094746 4.680792 8.947157 9.464206
[2,] 1.410008 5.752949 9.031771 5.962915 5.109533
[3,] 9.611500 5.080007 7.098136 6.153701 1.926322
[4,] 9.098425 3.214790 1.378536 3.951286 9.590533
[5,] 9.005854 7.235231 6.764561 9.948428 6.901352
[6,] 7.376774 5.896594 6.347278 3.602438 2.324023


### 3. Compute Full Euclidean Distance Matrix

In [4]:
dist_matrix <- as.matrix(dist(sample_feature_matrix, method = "euclidean"))

cat("\nEuclidean Distance Matrix:\n")
print(dist_matrix)


Euclidean Distance Matrix:
          1         2         3         4        5         6
1  0.000000  7.551716 10.762688  9.489792 6.480943 10.075246
2  7.551716  0.000000  9.034683 12.175879 9.172197  7.494021
3 10.762688  9.034683  0.000000 10.002343 6.653867  3.590503
4  9.489792 12.175879 10.002343  0.000000 9.401020  9.368480
5  6.480943  9.172197  6.653867  9.401020 0.000000  8.114401
6 10.075246  7.494021  3.590503  9.368480 8.114401  0.000000


 ### 4. Initialize Rank Matrix

In [5]:
rank_matrix <- matrix(0, nrow = S, ncol = S)

### 5. Rank the Distances

In [6]:
flat_dist <- dist_matrix
flat_dist[lower.tri(flat_dist, diag = TRUE)] <- NA  # Use only upper triangle
dist_vals <- na.omit(as.vector(flat_dist))
dist_ranks <- rank(dist_vals)

 ### 6.Fill the Rank Matrix
 #### Purpose: Fills the upper triangle of rank_matrix with ranks, then mirrors them to the lower triangle to make the matrix symmetric.

In [7]:
rank_matrix[upper.tri(rank_matrix)] <- dist_ranks
# Mirror to lower triangle for symmetry
rank_matrix <- rank_matrix + t(rank_matrix)

cat("\nFull Rank Matrix:\n")
print(rank_matrix)


Full Rank Matrix:
     [,1] [,2] [,3] [,4] [,5] [,6]
[1,]    0    5   14   11    2   13
[2,]    5    0    7   15    8    4
[3,]   14    7    0   12    3    1
[4,]   11   15   12    0   10    9
[5,]    2    8    3   10    0    6
[6,]   13    4    1    9    6    0


 ### 7. Extract In-Class and Out-Class Ranks
 #### Purpose: Separates ranks into two groups:

#### in_class_ranks: Ranks of distances between samples of the same class (R-R or G-G).

#### out_class_ranks: Ranks of distances between samples of different classes (R-G).

In [8]:
in_class_ranks <- c()
out_class_ranks <- c()

for (i in 1:(S - 1)) {
  for (j in (i + 1):S) {
    if (classes[i] == classes[j]) {
      in_class_ranks <- c(in_class_ranks, rank_matrix[i, j])
    } else {
      out_class_ranks <- c(out_class_ranks, rank_matrix[i, j])
    }
  }
}

cat("\nIn-Class Ranks (Red + Green within each):\n")
print(in_class_ranks)

cat("\nOut-Class Ranks (Between Red-Green):\n")
print(out_class_ranks)


In-Class Ranks (Red + Green within each):
[1]  5 14  7 10  9  6

Out-Class Ranks (Between Red-Green):
[1] 11  2 13 15  8  4 12  3  1


### 8.Display In-Class and Out-Class Rank Matrix
#### Purpose: Prints the rank matrix with:

#### * for in-class ranks (same class).

#### Plain numbers for out-class ranks (different classes).

#### Diagonal entries are 0 (distance to self).

In [9]:
cat("\nIn-Class and Out-Class Rank Matrix (textual):\n")
for (i in 1:S) {
  for (j in 1:S) {
    if (i == j) {
      cat("  0 ")
    } else if (classes[i] == classes[j]) {
      cat(sprintf(" %2d*", rank_matrix[i, j]))  # In-class with *
    } else {
      cat(sprintf(" %2d ", rank_matrix[i, j]))  # Out-class plain
    }
  }
  cat("\n")
}


In-Class and Out-Class Rank Matrix (textual):
  0   5* 14* 11   2  13 
  5*  0   7* 15   8   4 
 14*  7*  0  12   3   1 
 11  15  12   0  10*  9*
  2   8   3  10*  0   6*
 13   4   1   9*  6*  0 


### 9. Alpha Score Calculation

In [10]:
# Calculate M and N (for normalization)
m <- sum(classes == "R")  # Number of samples in class R
n <- sum(classes == "G")  # Number of samples in class G

M <- (m + n) * (m + n - 1) / 2  # Total possible pairs (full matrix)
N <- (m * (m - 1) + n * (n - 1)) / 2  # Total in-class pairs (R-R + G-G)

# Calculate A and B (theoretical min/max rank sums)
A <- sum(1:N)        # Minimal possible sum (best separation)
B <- sum((M - N + 1):M)  # Maximal possible sum (worst separation)

# Alpha score calculation
R_sum <- sum(in_class_ranks)  # Observed in-class rank sum
alpha <- (B - R_sum) / (B - A)

# Print results
cat("Total pairs (M):", M, "\n")
cat("In-class pairs (N):", N, "\n")
cat("Minimal rank sum (A):", A, "\n")
cat("Maximal rank sum (B):", B, "\n")
cat("Observed in-class rank sum (R_sum):", R_sum, "\n")
cat("Alpha score:", round(alpha, 4), "\n")

Total pairs (M): 15 
In-class pairs (N): 6 
Minimal rank sum (A): 21 
Maximal rank sum (B): 75 
Observed in-class rank sum (R_sum): 51 
Alpha score: 0.4444 
